Основные этапы решения

Задать входные параметры: количество автобусов, количество остановок, время полного маршрута, часы пик и пассажирный поток.

Создать классы: для автобусов, водителей и маршрута.

Планирование водителей: выбрать, сколько нужно 8-часовых и 12-часовых водителей в зависимости от расписания автобусов.

Составить расписание: распределить водителей по сменам и по времени.

Вывести результаты: сформировать таблицу расписания с учётом пересменок и времени на обед для 8-часовых водителей.


Жадный алгоритм

In [1]:
import random
from tkinter import *  # Для работы с графическим интерфейсом

# Глобальные переменные для водителей и маршрутов
drivers_A = ["Driver A1", "Driver A2", "Driver A3"]
drivers_B = ["Driver B1", "Driver B2", "Driver B3"]
route_times = [8, 9, 10, 11, 12, 13, 14, 15]  # Пример возможных начальных времен маршрутов
traffic_route_time = 1  # Время маршрута с учетом пробок (в часах)
shift_duration_A = 8  # Продолжительность смены для типа A (в часах)
shift_duration_B = 8  # Продолжительность смены для типа B (в часах)

# Функция проверки пересечения времени
def is_time_overlap(start1, end1, routes):
    for start2, end2 in routes:
        if (start1 < end2 and start2 < end1):
            return True  # Пересечение времени
    return False

# Функция вычисления времени окончания маршрута
def calculate_route_end(start_time, route_duration):
    return start_time + route_duration

# Функция для жадного планирования маршрутов
def greedy_schedule(driver_list, shift_duration, num_routes, traffic_route_time):
    schedule = {driver: [] for driver in driver_list}  # Создаем пустое расписание для каждого водителя
    
    # Генерация маршрутов для водителей
    all_routes = []
    for _ in range(num_routes):
        start_time = random.choice(route_times)
        end_time = calculate_route_end(start_time, traffic_route_time)
        all_routes.append((start_time, end_time))

    # Жадное распределение маршрутов между водителями
    for route in all_routes:
        # Находим водителя с наименьшим количеством пересечений
        best_driver = None
        best_penalty = float('inf')

        for driver, routes in schedule.items():
            # Проверяем на пересечения с уже назначенными маршрутами водителя
            if not is_time_overlap(route[0], route[1], routes):
                penalty = len(routes)  # Количество маршрутов водителя как "стоимость" (чем больше, тем хуже)
                if penalty < best_penalty:
                    best_penalty = penalty
                    best_driver = driver
        
        # Назначаем маршрут найденному водителю
        if best_driver is not None:
            schedule[best_driver].append(route)

    return schedule

# Отображение расписания в текстовом поле
def display_schedule(schedule, schedule_text):
    schedule_text.delete(1.0, END)
    for driver, routes in schedule.items():
        schedule_text.insert(END, f"Водитель: {driver}\n")
        for start, end in routes:
            schedule_text.insert(END, f"  Рейс с {start} до {end}\n")
        schedule_text.insert(END, "\n")

# Генерация расписания для водителей типа A
def generate_greedy_schedule_A(num_routes_entry, schedule_text):
    try:
        num_routes = int(num_routes_entry.get())
        if not drivers_A:
            schedule_text.insert(END, "\nНет водителей типа A для создания расписания.\n")
            return
        best_schedule = greedy_schedule(drivers_A, shift_duration_A, num_routes, traffic_route_time)
        display_schedule(best_schedule, schedule_text)
    except ValueError:
        schedule_text.insert(END, "\nОшибка: Введите корректное количество маршрутов.\n")

# Генерация расписания для водителей типа B
def generate_greedy_schedule_B(num_routes_entry, schedule_text):
    try:
        num_routes = int(num_routes_entry.get())
        if not drivers_B:
            schedule_text.insert(END, "\nНет водителей типа B для создания расписания.\n")
            return
        best_schedule = greedy_schedule(drivers_B, shift_duration_B, num_routes, traffic_route_time)
        display_schedule(best_schedule, schedule_text)
    except ValueError:
        schedule_text.insert(END, "\nОшибка: Введите корректное количество маршрутов.\n")

# Графический интерфейс для отображения кнопок и вывода расписания
def create_gui():
    root = Tk()
    root.title("Генератор расписания")

    global schedule_text
    schedule_text = Text(root, height=15, width=50)
    schedule_text.pack()

    num_routes_label = Label(root, text="Введите количество маршрутов:")
    num_routes_label.pack()

    num_routes_entry = Entry(root)
    num_routes_entry.pack()

    button_frame = Frame(root)
    button_frame.pack()

    greedy_button_A = Button(button_frame, text="Расписание (A)", command=lambda: generate_greedy_schedule_A(num_routes_entry, schedule_text), bg="white", fg="#3D0071", font=("Helvetica", 12), relief="solid", bd=2)
    greedy_button_A.pack(pady=5, fill="x")

    greedy_button_B = Button(button_frame, text="Расписание (Б)", command=lambda: generate_greedy_schedule_B(num_routes_entry, schedule_text), bg="white", fg="#3D0071", font=("Helvetica", 12), relief="solid", bd=2)
    greedy_button_B.pack(pady=5, fill="x")

    root.mainloop()

# Запуск графического интерфейса
create_gui()


Генетический алгоритм

In [3]:
import random
from tkinter import *  # Для работы с графическим интерфейсом

# Глобальные переменные для водителей и маршрутов
drivers_A = ["Driver A1", "Driver A2", "Driver A3"]
drivers_B = ["Driver B1", "Driver B2", "Driver B3"]
route_times = [8, 9, 10, 11, 12, 13, 14, 15]  # Пример возможных начальных времен маршрутов
traffic_route_time = 1  # Время маршрута с учетом пробок (в часах)
shift_duration_A = 8  # Продолжительность смены для типа A (в часах)
shift_duration_B = 8  # Продолжительность смены для типа B (в часах)

# Функция проверки пересечения времени
def is_time_overlap(start1, end1, routes):
    for start2, end2 in routes:
        if start1 < end2 and start2 < end1:
            return True  # Пересечение времени
    return False

# Функция вычисления времени окончания маршрута
def calculate_route_end(start_time, route_duration):
    return start_time + route_duration

# Генерация популяции расписаний
def create_population(driver_list, num_routes, population_size=100):
    population = []
    for _ in range(population_size):
        schedule = {driver: [] for driver in driver_list}
        used_times = []  # Отслеживаем уже занятые интервалы
        for _ in range(num_routes):
            driver = random.choice(driver_list)
            valid_time_found = False
            while not valid_time_found:
                start_time = random.choice(route_times)
                end_time = calculate_route_end(start_time, traffic_route_time)
                if not is_time_overlap(start_time, end_time, used_times):
                    schedule[driver].append((start_time, end_time))
                    used_times.append((start_time, end_time))
                    valid_time_found = True
        population.append(schedule)
    return population

# Скрещивание двух родителей
def crossover(parent1, parent2, driver_list):
    child = {driver: [] for driver in driver_list}
    for driver in driver_list:
        if random.random() > 0.5:
            child[driver] = parent1[driver]
        else:
            child[driver] = parent2[driver]
    return child

# Мутация расписания
def mutate(schedule, driver_list):
    driver = random.choice(driver_list)
    if schedule[driver]:
        schedule[driver].pop()  # Удаляем один рейс
    used_times = [time for routes in schedule.values() for time in routes]  # Собираем занятые интервалы
    valid_time_found = False
    while not valid_time_found:
        start_time = random.choice(route_times)
        end_time = calculate_route_end(start_time, traffic_route_time)
        if not is_time_overlap(start_time, end_time, used_times):
            schedule[driver].append((start_time, end_time))
            valid_time_found = True
    return schedule

# Генетический алгоритм для генерации расписания
def genetic_schedule(driver_list, shift_duration, num_routes, traffic_route_time, max_generations=50, population_size=100):
    population = create_population(driver_list, num_routes, population_size)
    
    for generation in range(max_generations):
        # Оцениваем популяцию
        population = sorted(population, key=lambda x: fitness(x), reverse=True)
        next_population = population[:10]  # Топ-10 лучших решений

        while len(next_population) < population_size:
            # Отбор двух случайных родителей
            parent1 = random.choice(population[:50])
            parent2 = random.choice(population[:50])

            # Скрещивание
            child = crossover(parent1, parent2, driver_list)

            # Мутация
            if random.random() < 0.2:
                child = mutate(child, driver_list)

            next_population.append(child)

        population = next_population

    # Возвращаем лучшее решение
    best_schedule = max(population, key=lambda x: fitness(x))
    return best_schedule

# Функция оценки расписания (fitness)
def fitness(schedule):
    penalties = 0
    all_routes = [time for routes in schedule.values() for time in routes]
    for i in range(len(all_routes)):
        for j in range(i + 1, len(all_routes)):
            if is_time_overlap(all_routes[i][0], all_routes[i][1], all_routes[j:j + 1]):
                penalties += 1
    return -penalties  # Меньше пересечений -> лучше

# Отображение расписания в текстовом поле
def display_schedule(schedule, schedule_text):
    schedule_text.delete(1.0, END)
    for driver, routes in schedule.items():
        schedule_text.insert(END, f"Водитель: {driver}\n")
        for start, end in routes:
            schedule_text.insert(END, f"  Рейс с {start} до {end}\n")
        schedule_text.insert(END, "\n")

# Генерация расписания для водителей типа A
def generate_genetic_schedule_A(num_routes_entry, schedule_text):
    try:
        num_routes = int(num_routes_entry.get())
        if not drivers_A:
            schedule_text.insert(END, "\nНет водителей типа A для создания расписания.\n")
            return
        best_schedule = genetic_schedule(drivers_A, shift_duration_A, num_routes, traffic_route_time)
        display_schedule(best_schedule, schedule_text)
    except ValueError:
        schedule_text.insert(END, "\nОшибка: Введите корректное количество маршрутов.\n")

# Генерация расписания для водителей типа B
def generate_genetic_schedule_B(num_routes_entry, schedule_text):
    try:
        num_routes = int(num_routes_entry.get())
        if not drivers_B:
            schedule_text.insert(END, "\nНет водителей типа B для создания расписания.\n")
            return
        best_schedule = genetic_schedule(drivers_B, shift_duration_B, num_routes, traffic_route_time)
        display_schedule(best_schedule, schedule_text)
    except ValueError:
        schedule_text.insert(END, "\nОшибка: Введите корректное количество маршрутов.\n")

# Графический интерфейс для отображения кнопок и вывода расписания
def create_gui():
    root = Tk()
    root.title("Генератор расписания")

    global schedule_text
    schedule_text = Text(root, height=15, width=50)
    schedule_text.pack()

    num_routes_label = Label(root, text="Введите количество маршрутов:")
    num_routes_label.pack()

    num_routes_entry = Entry(root)
    num_routes_entry.pack()

    button_frame = Frame(root)
    button_frame.pack()

    genetic_button_A = Button(button_frame, text="Генерасписание (A)", command=lambda: generate_genetic_schedule_A(num_routes_entry, schedule_text), bg="white", fg="#3D0071", font=("Helvetica", 12), relief="solid", bd=2)
    genetic_button_A.pack(pady=5, fill="x")

    genetic_button_B = Button(button_frame, text="Генерасписание (Б)", command=lambda: generate_genetic_schedule_B(num_routes_entry, schedule_text), bg="white", fg="#3D0071", font=("Helvetica", 12), relief="solid", bd=2)
    genetic_button_B.pack(pady=5, fill="x")

    root.mainloop()

# Запуск графического интерфейса
create_gui()
